In [1]:
import intersect
import pandas as pd
import geopandas as gpd
import fiona.crs
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

In [2]:
EPSG = 2263

CRS = {
    'proj': 'latlong',
    'init': 'epsg:{:d}'.format(EPSG)
}

In [ ]:
raw_people = pd.read_csv("/Users/asiega/Desktop/CVHpeople.csv")

In [ ]:
raw_people.head(2)

In [3]:
people = intersect.people_df("/Users/asiega/Desktop/CVHpeople.csv", crs=CRS)

In [4]:
people.head(2)

,Internal Contact ID,Latitude,Longitude,geometry
0,5,40.687482,-73.963384,POINT (-73.96338399999999 40.687482)
1,226,40.769909,-73.992111,POINT (-73.99211099999999 40.76990900000001)


In [5]:
def explode(indata):
    """Break down multipolygons in geojson to single polygons per row"""
    indf = gpd.GeoDataFrame.from_file(indata)
    outdf = gpd.GeoDataFrame(columns=indf.columns)
    for idx, row in indf.iterrows():
        if type(row.geometry) == Polygon:
            outdf = outdf.append(row,ignore_index=True)
        if type(row.geometry) == MultiPolygon:
            multdf = gpd.GeoDataFrame(columns=indf.columns)
            recs = len(row.geometry)
            multdf = multdf.append([row]*recs,ignore_index=True)
            for geom in range(recs):
                multdf.loc[geom,'geometry'] = row.geometry[geom]
            outdf = outdf.append(multdf,ignore_index=True)
    return outdf

# raw_shapes = explode("/Users/asiega/Desktop/NYCHA.geojson")

In [12]:
def shapes_df(path):
#     raw_shapes = gpd.read_file(path)
    raw_shapes = explode(path)
    raw_shapes.crs = fiona.crs.from_epsg(EPSG)
    zones = raw_shapes.to_crs(fiona.crs.from_epsg(EPSG))
    zones_crs = zones.to_crs(CRS)
    return zones_crs

In [13]:
shapes = shapes_df("/Users/asiega/Desktop/NYCHA.geojson")

In [18]:
people["geometry"].sort_values().head()

1        POINT (-73.99211099999999 40.76990900000001)
11421            POINT (-73.84563799999999 40.819266)
11420            POINT (-73.846222 40.81932800000001)
11419            POINT (-73.98685500000001 40.574812)
0                POINT (-73.96338399999999 40.687482)
Name: geometry, dtype: object

In [17]:
shapes["geometry"].sort_values().head()

851    POLYGON ((-73.79061904700808 40.66815582089683...
854    POLYGON ((-73.79838341966898 40.67181201846752...
852    POLYGON ((-73.80503351710118 40.66845837598144...
853    POLYGON ((-73.80219003598181 40.67066307272878...
850    POLYGON ((-73.80003796052573 40.59850693799366...
Name: geometry, dtype: object

In [19]:
len(shapes)

880

In [35]:
def merge_within(shapes, people):
    merged = gpd.sjoin(people, shapes, how='left', op='within')
    del merged['geometry']
    del merged['index_right']
    return merged.dropna()

In [36]:
merged_within = merge_within(shapes, people)

In [37]:
merged_within

,Internal Contact ID,Latitude,Longitude,BOROUGH,CUR_UNIT11,DEVELOPMEN,NONRES_BLD,RES_BLDG,TDS_NUM,TOT_POP11
89,912,40.717385,-73.978243,MANHATTAN,2194.0,BARUCH,1.0,17.0,060,5274.0
94,917,40.694176,-73.981118,BROOKLYN,1826.0,INGERSOLL,1.0,20.0,014,3227.0
168,1092,40.791862,-73.969814,MANHATTAN,70.0,WSUR (SITE A) 120 WEST 94TH STREET,0.0,1.0,151,154.0
169,1093,40.790859,-73.967413,MANHATTAN,40.0,REHAB PROGRAM (WISE REHAB),0.0,1.0,517,63.0
170,1094,40.884713,-73.843554,BRONX,2035.0,EDENWALD,2.0,40.0,057,5181.0
171,1095,40.847102,-73.887959,BRONX,219.0,TWIN PARKS EAST (SITE 9),0.0,1.0,287,229.0
202,1127,40.694918,-73.942191,BROOKLYN,84.0,BEDFORD-STUYVESANT REHAB,0.0,3.0,311,193.0
204,1129,40.704959,-73.941314,BROOKLYN,509.0,BORINQUEN PLAZA I,2.0,8.0,243,1183.0
205,1130,40.718419,-73.938480,BROOKLYN,700.0,COOPER PARK,0.0,11.0,069,1566.0
209,1134,40.694777,-73.982143,BROOKLYN,1826.0,INGERSOLL,1.0,20.0,014,3227.0


In [40]:
merged_within.to_csv("/Users/asiega/Desktop/preliminary_merge.csv")

In [22]:
merged.head()

,Internal Contact ID,Latitude,Longitude,BOROUGH,CUR_UNIT11,DEVELOPMEN,NONRES_BLD,RES_BLDG,TDS_NUM,TOT_POP11
0,5,40.687482,-73.963384,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,226,40.769909,-73.992111,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,228,40.746794,-73.982509,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,230,40.823687,-73.868344,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,232,40.760444,-73.973270,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
raw_shapes.crs = CRS
zones = raw_shapes.to_crs(people.crs)

In [ ]:
zones.head()

In [ ]:
places["geometry"].head()
# if you explore here, you notice that with to_crs, every polygon value becomes "inf"

In [ ]:
# this runs fine, but...
places.iloc[0]

In [ ]:
# this kills my kernel (Py root)
places.iloc[0]["geometry"]

In [ ]:
for geo in places["geometry"]:
    print geo[0]
    print "\n"
    print geo.bounds[0]
    break

### Compare json & geojson inputs

In [ ]:
raw_json = gpd.read_file("sample_data/shapes/nycha.json")

In [ ]:
raw_json["geometry"].head()

In [ ]:
raw_places["geometry"].head()

In [ ]:
# multipolygon vs regular polygon

raw_places["geometry"]

In [ ]:
EPSG = 2263

CRS = {
    'proj': 'latlong',
    'init': 'epsg:{:d}'.format(EPSG)
}

zones = exploded_df.to_crs(fiona.crs.from_epsg(EPSG))
zones_crs = zones.to_crs(CRS)

In [ ]:
zones_crs.head()

In [ ]:
exp_head = exploded_df.head(50)

merged_df = intersect.merge(exp_head, people)